# NER vs LLM

---

### Configure OLLama

Install OLLama

Open terminal and type:

`ollama run SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M`


In [1]:
import json

import pandas as pd
import requests

# This code returns an empty string...


def chat_ollama(text):
    url = "http://localhost:11434/api/chat"

    payload = {
        "model": "SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M",
        "messages": [{"role": "user", "content": text}],
        "stream": False,
    }

    response = requests.post(url, json=payload)

    # Parse the response
    if response.status_code == 200:
        return response.json()["message"]["content"]

    return None


question = "Możesz opowiedzieć mi o problemie Named Entity Recognition?"

chat_ollama(question)

'Named Entity Recognition (NER) to zadanie przetwarzania języka naturalnego, które polega na identyfikowaniu i klasyfikowaniu jednostek nazwanych w tekście. Jednostki nazwane to fragmenty tekstu, które reprezentują konkretne byty lub pojęcia, takie jak osoby, organizacje, lokalizacje, daty, ilości i inne specjalne terminy. NER jest kluczowym elementem wielu aplikacji związanych z analizą tekstu, takich jak ekstrakcja informacji, tłumaczenie maszynowe, systemy rekomendacyjne i wiele innych.\r\n\r\nProblem Named Entity Recognition polega na tym, że teksty w języku naturalnym są pełne niejednoznaczności i kontekstowych zależności. Słowa mogą mieć różne znaczenia w zależności od kontekstu, a jednostki nazwane mogą być wyrażone na wiele sposobów. Na przykład, słowo "John" może oznaczać osobę lub być skrótem od liczby 900 (jak w zapisie daty 1864-03-15). Podobnie, nazwa miasta "Warszawa" może być napisana jako "Warsaw" lub "Varsovie" w różnych językach i kontekstach. \r\n\r\nDodatkowo, NER m

In [2]:
url = "http://localhost:11434/api/generate"


def prompt_ollama(text):
    payload = {
        "model": "SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M",
        "prompt": text,
        "context": [],
        "options": {"top_k": 10, "temperature": 0},
        "stream": False,
    }

    response = requests.post(url, data=json.dumps(payload))

    # Parse the response
    if response.status_code == 200:
        return response.json()["response"]

    return None

In [3]:
prompt_ollama(
    "Identify so called named entities in this sentence: 'Kraków jest największym miastem w Polsce'. After that list out the entities in a form of strings in a python list. I want your response to only include the python list without any additional code or \"```\" characters. Be careful to use the polish form of the words."
)

"['Kraków', 'miastem', 'Polsce']"

In [4]:
prompt_ollama("Czy potrafisz mówić po polsku?")

'Tak, potrafię mówić po polsku. W czym mogę Ci pomóc?'

### Take 1000 passages from fiqa corpus

In [5]:
from datasets import load_dataset
from numpy.random import choice

fiqa_corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")["corpus"]

fiqa_corpus = fiqa_corpus["text"]
fiqa_idx = choice(len(fiqa_corpus), 100, replace=False)
fiqa_corpus = [fiqa_corpus[i] for i in fiqa_idx]

---

### NER baseline

In [6]:
import spacy

nlp = spacy.load("pl_core_news_sm")


def get_ents(text):
    doc = nlp(text)
    entity_dict = {}
    for ent in doc.ents:
        text, label = ent.text, ent.label_
        if (text, label) not in entity_dict.keys():
            entity_dict[(text, label)] = 0
        entity_dict[(text, label)] += 1
    return entity_dict

In [7]:
prompt = (
    "Chcę, abyś wymienił encje nazwane w polskim tekście, który Ci podam, i przypisał je do jednej z 6 kategorii."
    "Encje nazwane (eng. named entities) określają wybraną charakterystyczną jednostkę za pomocą zdefiniowanej nazwy."
    "6 kategorii, do których powinieneś przypisać encje nazwane: \n"
    "date - format daty/czasu: precyzyjna data, rok, itp. np. 11 Września. \n"
    "geogName - charakterystyczne obiekty geograficzne: rzeki, góry, itp. np. Wisła, Giewont. \n"
    "orgName - pełne nazwy organizacji: nazwy firm, itp. np. Microsoft, Bank PKO. "
    "persName - znane osoby: politycy, celebryci, postacie historyczne itp. np. Artur Rojek, Papież Franciszek. \n"
    "placeName - jednostki polityczne: miasta, państwa, itd. np. Kraków, Czechy, Australia, Bojszowy Nowe. \n"
    "time - Godzina. np. 18:30. \n"
    "Zignoruj fragmenty tekstu nie będące charakterystycznymi encjami nazwanymi. "
    "Wynik powinien być listą krotek. Każda krotka powinna składać się z encji nazwanej i odpowiadającej jej kategorii (jednej z wymienionych). "
    "Przykładowy format poprawnego wyniku dla tekstu: \n"
    '"Ostatnim razem w górach izerskich byłem 11 września": [("góry izerskie", "geogName"), ("11 września", "date")] \n'
    '"Nigdy nie byłem w stanach zjednoczonych, ale wiem kto to Barack Obama": [("Stany Zjednoczone", "placeName"), ("Barack Obama", "persName")] \n'
    "Nie dodawaj żadnych dodatkowych informacji, wyjaśnień, ani kodu w swojej odpowiedzi. Zwróć jedynie poprawną listą języka Python. "
    "Tekst: "
)

In [8]:
import re


def extract_list(text):
    regex = r"\[[^\[\]]*\]"
    return re.findall(regex, text)

In [37]:
from ast import literal_eval


# We use spacy to lemmatize if possible as the llm seems to be oblivious to the intricacy of case declension
def get_llm_ents(text):
    response = chat_ollama(prompt + text)

    print(response)

    lists = extract_list(response)
    if not lists or len(lists[0]) == 0:
        return {}
    response = lists[0]

    entity_dict = {}
    response = literal_eval(response)
    if len(response) == 0 or len(response[0]) != 2:
        return {}
    for entity, label in response:
        ents_tmp = list(nlp(entity).sents)
        if len(ents_tmp):
            lemma = " ".join([l.lemma_ for l in ents_tmp[0]])
        else:
            lemma = entity
        if (lemma, label) not in entity_dict.keys():
            entity_dict[(lemma, label)] = 0
        entity_dict[(lemma, label)] += 1
    return entity_dict

In [10]:
example_text = "Wczoraj w Krakowie miało miejsce spotkanie prezydentów Polski i Stanów Zjednoczonych. Polacy cieszyli się z wizyty Michaela Jacksona"

get_llm_ents(example_text)

[("Kraków", "placeName"), ("Polska", "placeName"), ("Stany Zjednoczone", "placeName"), ("Michael Jackson", "persName")]


{('Kraków', 'placeName'): 1,
 ('Polska', 'placeName'): 1,
 ('Stany Zjednoczone', 'placeName'): 1,
 ('Michael Jackson', 'persName'): 1}

---

### Compare spaCy to PHI-3

I gave up on trying to run it without providing the examples.

The returned results were so bad that parsing the outputs was practically impossible.

It's safe to conclude that few shot learning can drastically improve the quality of the response.

In [11]:
from pathlib import Path
import pickle

save_path = Path("data")
save_path.mkdir(exist_ok=True, parents=True)

In [12]:
from tqdm import tqdm

if not (save_path / "fiqa_spacy.pkl").exists():
    fiqa_spacy = [get_ents(text) for text in tqdm(fiqa_corpus)]

    with open(save_path / "fiqa_spacy.pkl", "wb") as fp:
        pickle.dump(fiqa_spacy, fp)

with open(save_path / "fiqa_spacy.pkl", "rb") as fp:
    fiqa_spacy = pickle.load(fp)

In [13]:
if not (save_path / "fiqa_llm.pkl").exists():
    fiqa_llm = []
    for text in tqdm(fiqa_corpus):
        fiqa_llm.append(get_llm_ents(text))

    with open(save_path / "fiqa_llm.pkl", "wb") as fp:
        pickle.dump(fiqa_llm, fp)

with open(save_path / "fiqa_llm.pkl", "rb") as fp:
    fiqa_llm = pickle.load(fp)

In [14]:
# with open(save_path / 'fiqa_llm_few_shot.pkl', 'wb') as fp:
#     pickle.dump(fiqa_llm_few_shot, fp)

In [15]:
def get_ent_cat_count(entity_dict_list):
    entities, categories = {}, {}
    for d in entity_dict_list:
        for (entity, category), count in d.items():
            if entity not in entities.keys():
                entities[entity] = 0
            if category not in categories.keys():
                categories[category] = 0
            entities[entity] += count
            categories[category] += count
    entities = sorted(list(entities.items()), key=lambda x: x[1], reverse=True)
    categories = sorted(list(categories.items()), key=lambda x: x[1], reverse=True)
    return entities, categories

In [16]:
spacy_entities, spacy_categories = get_ent_cat_count(fiqa_spacy)
llm_entities, llm_categories = get_ent_cat_count(fiqa_llm)
# few_shot_entities, few_shot_categories = get_ent_cat_count(fiqa_llm_few_shot)

In [17]:
spacy_categories

[('orgName', 141),
 ('placeName', 70),
 ('persName', 66),
 ('date', 18),
 ('geogName', 16)]

In [18]:
spacy_entities[:25]

[('CFA', 23),
 ('MBA', 16),
 ('USA', 12),
 ('Google', 5),
 ('HSA', 5),
 ('Indiach', 4),
 ('SS', 4),
 ('LLC', 4),
 ('eBay', 3),
 ('Forex', 3),
 ('IRS', 3),
 ('SIP', 3),
 ('Amazon', 3),
 ('CFD', 3),
 ('Apple', 3),
 ('Sydney', 3),
 ('CBV', 3),
 ('Stanach Zjednoczonych', 2),
 ('TPS', 2),
 ('amerykańskimi', 2),
 ('Wielkiej Brytanii', 2),
 ('BB', 2),
 ('Columbia', 2),
 ('Whartona', 2),
 ('Aptery', 2)]

In [19]:
llm_categories

[('orgName', 123),
 ('date', 55),
 ('placeName', 32),
 ('time', 17),
 ('persName', 15),
 ('geogName', 6),
 ('service', 4),
 ('currency', 2),
 ('', 2),
 ('object', 2),
 ('number', 1)]

In [20]:
llm_entities[:25]

[('1987', 33),
 ('Stany Zjednoczone', 7),
 ('wielki Brytania', 3),
 ('Barack Obama', 2),
 ('Amazon', 2),
 ('USA', 2),
 ('rezerwa Federalny', 2),
 ('UPS', 2),
 ('USPS', 2),
 ('fałszywy równoważność', 1),
 ('być prawie pewny', 1),
 ('dane', 1),
 ('lewicow źródło', 1),
 ('przeciwnik', 1),
 ('naukowy', 1),
 ('Mindwin', 1),
 ('rynek rolny', 1),
 ('rynek towarowy', 1),
 ('uniwersytet', 1),
 ('kawiarnia', 1),
 ('bank godzina', 1),
 ('Fort Lauderdale', 1),
 ('elektrownia na paliwo kopalny', 1),
 ('Wall Street', 1),
 ('Brexit', 1)]

We can see that the results of llm retrieval is horrible in comparison to spaCy's ner. The model hallucinated several new classes and has numerous false positives.

In [66]:
import pandas as pd

df = pd.read_csv("annotations.csv")

In [63]:
class_map = {
    "PER": "persName",
    "LOC": "placeName",
    "ORG": "orgName",
    "GPE": "geogName",
    "DATE": "date",
    "TIME": "time",
}


def extract_from_annotation(annotation):
    entity_dict = {}
    for entity, category, _ in annotation:
        ents_tmp = list(nlp(entity).sents)
        if len(ents_tmp):
            entity = " ".join([l.lemma_ for l in ents_tmp[0]])

        category = class_map[category]
        if (entity, category) not in entity_dict.keys():
            entity_dict[(entity, category)] = 0
        entity_dict[(entity, category)] += 1
    return entity_dict

In [68]:
texts = df["Tekst"]
entities = [extract_from_annotation(literal_eval(e)) for e in df["Encje"]]

In [38]:
from tqdm import tqdm

if not (save_path / "predictions.pkl").exists():
    predictions = []

    for text in tqdm(texts):
        predictions.append(get_llm_ents(text))

    with open(save_path / "predictions.pkl", "wb") as fp:
        pickle.dump(predictions, fp)

with open(save_path / "predictions.pkl", "rb") as fp:
    predictions = pickle.load(fp)

  5%|▌         | 1/19 [00:03<00:56,  3.16s/it]

[("Jake Paul", "persName"), ("Mike Tyson", "persName"), ("AT&T Stadium", "orgName"), ("Arlington", "placeName"), ("Teksas", "placeName")]


 11%|█         | 2/19 [00:06<00:50,  2.99s/it]

[("Ridley Scott", "persName"), ("pierwsza część", "date"), ("Acacius", "persName"), ("Pedro Pascal", "persName")]


 16%|█▌        | 3/19 [00:08<00:46,  2.88s/it]

...

[("Chiny", "orgName"), ("Rosja", "orgName"), ("Korea Południowa", "placeName")]


 95%|█████████▍| 18/19 [00:51<00:02,  2.84s/it]

[("Gladiator 2", "orgName"), ("Paul Mescal", "persName"), ("Królestwo niebieskie", "placeName"), ("Orlando Bloom", "persName")]


100%|██████████| 19/19 [00:54<00:00,  2.89s/it]

[("Yi Peng", "orgName"), ("duńskie służby", "orgName"), ("cieśnina Kattegat", "geogName"), ("Chiny", "placeName")]


In [74]:
def get_TP_FP_FN(y_test, y_pred, use_class=False):
    if not use_class:
        y_test = {k[0]: v for k, v in y_test.items()}
        y_pred = {k[0]: v for k, v in y_pred.items()}

    TP = FP = FN = 0

    for e in y_pred:
        if e in y_test:
            pred = y_pred[e]
            test = y_test[e]
            if pred == test:
                TP += pred
            elif pred > test:
                TP += test
                FP += pred - test
            else:
                TP += pred
                FN += test - pred
        else:
            FP += y_pred[e]

    for e in y_test:
        if e not in y_pred:
            FN += y_test[e]

    return TP, FP, FN

In [75]:
scores = [get_TP_FP_FN(t, p) for t, p in zip(entities, predictions)]

In [77]:
TP = sum([scr[0] for scr in scores])
FP = sum([scr[1] for scr in scores])
FN = sum([scr[2] for scr in scores])

In [80]:
TP, FP, FN

(58, 11, 36)

In [82]:
prec = TP / (TP + FP)
rec = TP / (TP + FN)
f1 = 2 * (prec * rec) / (prec + rec)

In [83]:
print(f"Precission : {prec}")
print(f"Recall     : {rec}")
print(f"f1 score   : {f1}")

Precission : 0.8405797101449275
Recall     : 0.6170212765957447
f1 score   : 0.7116564417177915


The model skips around 40% of entities it should be detecting but it only 15% of it's predictions are false. 

I'm surprised by how well it worked